
<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning">
</div>


# Lab - Creating Bronze Tables from JSON Files
### Duration: ~ 15 minutes

In this lab you will ingest a JSON file as Delta table and then flatten the JSON formatted string column.

### Learning Objectives
  - Inspect a raw JSON file.
  - Read in JSON files to a Delta table and flatten the JSON formatted string column.

## REQUIRED - SELECT CLASSIC COMPUTE

Before executing cells in this notebook, please select your classic compute cluster in the lab. Be aware that **Serverless** is enabled by default and you have a Shared SQL warehouse.

<!-- ![Select Cluster](./Includes/images/selecting_cluster_info.png) -->

Follow these steps to select the classic compute cluster:


1. Navigate to the top-right of this notebook and click the drop-down menu to select your cluster. By default, the notebook will use **Serverless**.

2. If your cluster is available, select it and continue to the next cell. If the cluster is not shown:

   - Click **More** in the drop-down.

   - In the **Attach to an existing compute resource** window, use the first drop-down to select your unique cluster.

**NOTE:** If your cluster has terminated, you might need to restart it in order to select it. To do this:

1. Right-click on **Compute** in the left navigation pane and select *Open in new tab*.

2. Find the triangle icon to the right of your compute cluster name and click it.

3. Wait a few minutes for the cluster to start.

4. Once the cluster is running, complete the steps above to select your cluster.


## A. Classroom Setup

Run the following cell to configure your working environment for this notebook.

**NOTE:** The `DA` object is only used in Databricks Academy courses and is not available outside of these courses. It will dynamically reference the information needed to run the course in the lab environment.

In [0]:
%run ./Includes/Classroom-Setup-07L

Note: you may need to restart the kernel using %restart_python or dbutils.library.restartPython() to use updated packages.


----------------------------------------------------------------------------------------
Directory /Volumes/dbacademy/ops/labuser10989365_1753272590@vocareum_com/csv_demo_files already exists. No action taken.
Directory /Volumes/dbacademy/ops/labuser10989365_1753272590@vocareum_com/json_demo_files already exists. No action taken.
Directory /Volumes/dbacademy/ops/labuser10989365_1753272590@vocareum_com/xml_demo_files already exists. No action taken.
----------------------------------------------------------------------------------------



Generated file: /Volumes/dbacademy/ops/labuser10989365_1753272590@vocareum_com/json_demo_files/lab_kafka_events.json


File written with 1 malformed 'timestamp' at position 64: /Volumes/dbacademy/ops/labuser10989365_1753272590@vocareum_com/json_demo_files/lab_kafka_events_challenge.json


Run the cell below to view your default catalog and schema. Notice that your default catalog is **dbacademy** and your default schema is your unique **labuser** schema.

**NOTE:** The default catalog and schema are pre-configured for you to avoid the need to specify the three-level name for when writing your tables (i.e., catalog.schema.table).

In [0]:
SELECT current_catalog(), current_schema()

current_catalog(),current_schema()
dbacademy,labuser10989365_1753272590


## B. Lab - JSON Ingestion
**Scenario:** You are working with your data team on ingesting a JSON file into Databricks. Your job is to ingest the JSON file as is into a bronze table, then create a second bronze table that flattens the JSON formatted string column in the raw bronze table for downstream processing.

### B1. Inspect the Dataset

1. In the cell below, view the value of the Python variable `DA.paths.working_dir`. This variable will reference your **dbacademy.ops.labuser** volume, as each user has a different source volume. This variable is created within the classroom setup script to dynamically reference your unique volume.

   Run the cell and review the results. You’ll notice that the `DA.paths.working_dir` variable points to your `/Volumes/dbacademy/ops/labuser` volume.

**Note:** Instead of using the `DA.paths.working_dir` variable, you could also specify the path name directly by right clicking on your volume and selecting **Copy volume path**.

In [0]:
%python
print(DA.paths.working_dir)

/Volumes/dbacademy/ops/labuser10989365_1753272590@vocareum_com


Run the cell to view the data in the `/Volumes/dbacademy/ops/your-labuser-name/json_demo_files/lab_kafka_events.json` file in the location from above.

In [0]:
%python
spark.sql(f'''
          SELECT * 
          FROM json.`{DA.paths.working_dir}/json_demo_files/lab_kafka_events.json`
          ''').display()

key,timestamp,value
event_0,1.753283167516484E9,eyJ1c2VyX2lkIjogInVzZXJfNDQzMCIsICJldmVudF90eXBlIjogInZpZXciLCAiZXZlbnRfdGltZXN0YW1wIjogIjIwMjUtMDctMjNUMTU6MDY6MDcuNTE2NDMyIiwgIml0ZW1zIjogW3siaXRlbV9pZCI6ICJpdGVtXzgxMyIsICJxdWFudGl0eSI6IDEsICJwcmljZV91c2QiOiAzOS4zNn0sIHsiaXRlbV9pZCI6ICJpdGVtXzExMiIsICJxdWFudGl0eSI6IDQsICJwcmljZV91c2QiOiA0MS4xMn1dfQ==
event_1,1.753283167516559E9,eyJ1c2VyX2lkIjogInVzZXJfNTQ4NCIsICJldmVudF90eXBlIjogImNsaWNrIiwgImV2ZW50X3RpbWVzdGFtcCI6ICIyMDI1LTA3LTIzVDE1OjA2OjA3LjUxNjQ5MyIsICJpdGVtcyI6IFt7Iml0ZW1faWQiOiAiaXRlbV82ODgiLCAicXVhbnRpdHkiOiAxLCAicHJpY2VfdXNkIjogMjEuNjh9LCB7Iml0ZW1faWQiOiAiaXRlbV80MjMiLCAicXVhbnRpdHkiOiAyLCAicHJpY2VfdXNkIjogMjIuODJ9LCB7Iml0ZW1faWQiOiAiaXRlbV8yODIiLCAicXVhbnRpdHkiOiAzLCAicHJpY2VfdXNkIjogOTMuNDZ9XX0=
event_2,1.753283167516581E9,eyJ1c2VyX2lkIjogInVzZXJfNjkzNSIsICJldmVudF90eXBlIjogInB1cmNoYXNlIiwgImV2ZW50X3RpbWVzdGFtcCI6ICIyMDI1LTA3LTIzVDE1OjA2OjA3LjUxNjU2NCIsICJpdGVtcyI6IFt7Iml0ZW1faWQiOiAiaXRlbV84MjgiLCAicXVhbnRpdHkiOiAyLCAicHJpY2VfdXNkIjogMTQuNTN9XX0=
event_3,1.753283167516599E9,eyJ1c2VyX2lkIjogInVzZXJfMjYyNyIsICJldmVudF90eXBlIjogInB1cmNoYXNlIiwgImV2ZW50X3RpbWVzdGFtcCI6ICIyMDI1LTA3LTIzVDE1OjA2OjA3LjUxNjU4NCIsICJpdGVtcyI6IFt7Iml0ZW1faWQiOiAiaXRlbV85OTkiLCAicXVhbnRpdHkiOiA1LCAicHJpY2VfdXNkIjogOTYuMjN9XX0=
event_4,1.753283167516619E9,eyJ1c2VyX2lkIjogInVzZXJfNDQ0NiIsICJldmVudF90eXBlIjogInZpZXciLCAiZXZlbnRfdGltZXN0YW1wIjogIjIwMjUtMDctMjNUMTU6MDY6MDcuNTE2NjAzIiwgIml0ZW1zIjogW3siaXRlbV9pZCI6ICJpdGVtXzQ4NyIsICJxdWFudGl0eSI6IDUsICJwcmljZV91c2QiOiA1MC45OX1dfQ==
event_5,1.753283167516643E9,eyJ1c2VyX2lkIjogInVzZXJfMzM0NSIsICJldmVudF90eXBlIjogInZpZXciLCAiZXZlbnRfdGltZXN0YW1wIjogIjIwMjUtMDctMjNUMTU6MDY6MDcuNTE2NjIzIiwgIml0ZW1zIjogW3siaXRlbV9pZCI6ICJpdGVtXzY0OSIsICJxdWFudGl0eSI6IDEsICJwcmljZV91c2QiOiAxNi4xM30sIHsiaXRlbV9pZCI6ICJpdGVtXzY4NCIsICJxdWFudGl0eSI6IDQsICJwcmljZV91c2QiOiA0OC43Nn0sIHsiaXRlbV9pZCI6ICJpdGVtXzYyNSIsICJxdWFudGl0eSI6IDQsICJwcmljZV91c2QiOiA4NC4wNH1dfQ==
event_6,1.753283167516666E9,eyJ1c2VyX2lkIjogInVzZXJfOTY0MSIsICJldmVudF90eXBlIjogInZpZXciLCAiZXZlbnRfdGltZXN0YW1wIjogIjIwMjUtMDctMjNUMTU6MDY6MDcuNTE2NjQ1IiwgIml0ZW1zIjogW3siaXRlbV9pZCI6ICJpdGVtXzk4MCIsICJxdWFudGl0eSI6IDQsICJwcmljZV91c2QiOiA4MC4wMn0sIHsiaXRlbV9pZCI6ICJpdGVtXzc2OSIsICJxdWFudGl0eSI6IDMsICJwcmljZV91c2QiOiAzMy4zNn0sIHsiaXRlbV9pZCI6ICJpdGVtXzU1MCIsICJxdWFudGl0eSI6IDQsICJwcmljZV91c2QiOiAzNy4zN31dfQ==
event_7,1.753283167516678E9,eyJ1c2VyX2lkIjogInVzZXJfNjEyOCIsICJldmVudF90eXBlIjogImNsaWNrIiwgImV2ZW50X3RpbWVzdGFtcCI6ICIyMDI1LTA3LTIzVDE1OjA2OjA3LjUxNjY2OCIsICJpdGVtcyI6IFt7Iml0ZW1faWQiOiAiaXRlbV8yNDMiLCAicXVhbnRpdHkiOiA1LCAicHJpY2VfdXNkIjogMzcuMDJ9XX0=
event_8,1.753283167516694E9,eyJ1c2VyX2lkIjogInVzZXJfODA5NiIsICJldmVudF90eXBlIjogInZpZXciLCAiZXZlbnRfdGltZXN0YW1wIjogIjIwMjUtMDctMjNUMTU6MDY6MDcuNTE2NjgwIiwgIml0ZW1zIjogW3siaXRlbV9pZCI6ICJpdGVtXzY4NiIsICJxdWFudGl0eSI6IDQsICJwcmljZV91c2QiOiA1NC4zfSwgeyJpdGVtX2lkIjogIml0ZW1fNzYyIiwgInF1YW50aXR5IjogNCwgInByaWNlX3VzZCI6IDEzLjY1fSwgeyJpdGVtX2lkIjogIml0ZW1fODI0IiwgInF1YW50aXR5IjogMSwgInByaWNlX3VzZCI6IDY2LjcyfV19
event_9,1.75328316751671E9,eyJ1c2VyX2lkIjogInVzZXJfNjcyMyIsICJldmVudF90eXBlIjogInZpZXciLCAiZXZlbnRfdGltZXN0YW1wIjogIjIwMjUtMDctMjNUMTU6MDY6MDcuNTE2Njk2IiwgIml0ZW1zIjogW3siaXRlbV9pZCI6ICJpdGVtXzkxMSIsICJxdWFudGl0eSI6IDIsICJwcmljZV91c2QiOiAyOC4wNX0sIHsiaXRlbV9pZCI6ICJpdGVtXzk4NiIsICJxdWFudGl0eSI6IDQsICJwcmljZV91c2QiOiA5NC43MX0sIHsiaXRlbV9pZCI6ICJpdGVtXzIwNyIsICJxdWFudGl0eSI6IDMsICJwcmljZV91c2QiOiAzMC4zOX1dfQ==


### B2. Create the Raw Bronze Table

Inspect and run the code below to ingest the raw JSON file `/Volumes/dbacademy/ops/your-labuser-name/json_demo_files/lab_kafka_events.json` and create the **lab7_lab_kafka_events_raw** table.

Notice the following:
- The **value** column is decoded.
- The **decoded_value** column was created and returns the decoded column as a JSON-formatted string.


In [0]:
select *
from text.`/Volumes/dbacademy/ops/labuser10989365_1753272590@vocareum_com/json_demo_files/lab_kafka_events.json`
-- from json.`/Volumes/dbacademy/ops/labuser10989365_1753272590@vocareum_com/json_demo_files/lab_kafka_events.json`

value
"{""key"": ""event_0"", ""timestamp"": 1753283167.516484, ""value"": ""eyJ1c2VyX2lkIjogInVzZXJfNDQzMCIsICJldmVudF90eXBlIjogInZpZXciLCAiZXZlbnRfdGltZXN0YW1wIjogIjIwMjUtMDctMjNUMTU6MDY6MDcuNTE2NDMyIiwgIml0ZW1zIjogW3siaXRlbV9pZCI6ICJpdGVtXzgxMyIsICJxdWFudGl0eSI6IDEsICJwcmljZV91c2QiOiAzOS4zNn0sIHsiaXRlbV9pZCI6ICJpdGVtXzExMiIsICJxdWFudGl0eSI6IDQsICJwcmljZV91c2QiOiA0MS4xMn1dfQ==""}"
"{""key"": ""event_1"", ""timestamp"": 1753283167.516559, ""value"": ""eyJ1c2VyX2lkIjogInVzZXJfNTQ4NCIsICJldmVudF90eXBlIjogImNsaWNrIiwgImV2ZW50X3RpbWVzdGFtcCI6ICIyMDI1LTA3LTIzVDE1OjA2OjA3LjUxNjQ5MyIsICJpdGVtcyI6IFt7Iml0ZW1faWQiOiAiaXRlbV82ODgiLCAicXVhbnRpdHkiOiAxLCAicHJpY2VfdXNkIjogMjEuNjh9LCB7Iml0ZW1faWQiOiAiaXRlbV80MjMiLCAicXVhbnRpdHkiOiAyLCAicHJpY2VfdXNkIjogMjIuODJ9LCB7Iml0ZW1faWQiOiAiaXRlbV8yODIiLCAicXVhbnRpdHkiOiAzLCAicHJpY2VfdXNkIjogOTMuNDZ9XX0=""}"
"{""key"": ""event_2"", ""timestamp"": 1753283167.516581, ""value"": ""eyJ1c2VyX2lkIjogInVzZXJfNjkzNSIsICJldmVudF90eXBlIjogInB1cmNoYXNlIiwgImV2ZW50X3RpbWVzdGFtcCI6ICIyMDI1LTA3LTIzVDE1OjA2OjA3LjUxNjU2NCIsICJpdGVtcyI6IFt7Iml0ZW1faWQiOiAiaXRlbV84MjgiLCAicXVhbnRpdHkiOiAyLCAicHJpY2VfdXNkIjogMTQuNTN9XX0=""}"
"{""key"": ""event_3"", ""timestamp"": 1753283167.516599, ""value"": ""eyJ1c2VyX2lkIjogInVzZXJfMjYyNyIsICJldmVudF90eXBlIjogInB1cmNoYXNlIiwgImV2ZW50X3RpbWVzdGFtcCI6ICIyMDI1LTA3LTIzVDE1OjA2OjA3LjUxNjU4NCIsICJpdGVtcyI6IFt7Iml0ZW1faWQiOiAiaXRlbV85OTkiLCAicXVhbnRpdHkiOiA1LCAicHJpY2VfdXNkIjogOTYuMjN9XX0=""}"
"{""key"": ""event_4"", ""timestamp"": 1753283167.516619, ""value"": ""eyJ1c2VyX2lkIjogInVzZXJfNDQ0NiIsICJldmVudF90eXBlIjogInZpZXciLCAiZXZlbnRfdGltZXN0YW1wIjogIjIwMjUtMDctMjNUMTU6MDY6MDcuNTE2NjAzIiwgIml0ZW1zIjogW3siaXRlbV9pZCI6ICJpdGVtXzQ4NyIsICJxdWFudGl0eSI6IDUsICJwcmljZV91c2QiOiA1MC45OX1dfQ==""}"
"{""key"": ""event_5"", ""timestamp"": 1753283167.516643, ""value"": ""eyJ1c2VyX2lkIjogInVzZXJfMzM0NSIsICJldmVudF90eXBlIjogInZpZXciLCAiZXZlbnRfdGltZXN0YW1wIjogIjIwMjUtMDctMjNUMTU6MDY6MDcuNTE2NjIzIiwgIml0ZW1zIjogW3siaXRlbV9pZCI6ICJpdGVtXzY0OSIsICJxdWFudGl0eSI6IDEsICJwcmljZV91c2QiOiAxNi4xM30sIHsiaXRlbV9pZCI6ICJpdGVtXzY4NCIsICJxdWFudGl0eSI6IDQsICJwcmljZV91c2QiOiA0OC43Nn0sIHsiaXRlbV9pZCI6ICJpdGVtXzYyNSIsICJxdWFudGl0eSI6IDQsICJwcmljZV91c2QiOiA4NC4wNH1dfQ==""}"
"{""key"": ""event_6"", ""timestamp"": 1753283167.516666, ""value"": ""eyJ1c2VyX2lkIjogInVzZXJfOTY0MSIsICJldmVudF90eXBlIjogInZpZXciLCAiZXZlbnRfdGltZXN0YW1wIjogIjIwMjUtMDctMjNUMTU6MDY6MDcuNTE2NjQ1IiwgIml0ZW1zIjogW3siaXRlbV9pZCI6ICJpdGVtXzk4MCIsICJxdWFudGl0eSI6IDQsICJwcmljZV91c2QiOiA4MC4wMn0sIHsiaXRlbV9pZCI6ICJpdGVtXzc2OSIsICJxdWFudGl0eSI6IDMsICJwcmljZV91c2QiOiAzMy4zNn0sIHsiaXRlbV9pZCI6ICJpdGVtXzU1MCIsICJxdWFudGl0eSI6IDQsICJwcmljZV91c2QiOiAzNy4zN31dfQ==""}"
"{""key"": ""event_7"", ""timestamp"": 1753283167.516678, ""value"": ""eyJ1c2VyX2lkIjogInVzZXJfNjEyOCIsICJldmVudF90eXBlIjogImNsaWNrIiwgImV2ZW50X3RpbWVzdGFtcCI6ICIyMDI1LTA3LTIzVDE1OjA2OjA3LjUxNjY2OCIsICJpdGVtcyI6IFt7Iml0ZW1faWQiOiAiaXRlbV8yNDMiLCAicXVhbnRpdHkiOiA1LCAicHJpY2VfdXNkIjogMzcuMDJ9XX0=""}"
"{""key"": ""event_8"", ""timestamp"": 1753283167.516694, ""value"": ""eyJ1c2VyX2lkIjogInVzZXJfODA5NiIsICJldmVudF90eXBlIjogInZpZXciLCAiZXZlbnRfdGltZXN0YW1wIjogIjIwMjUtMDctMjNUMTU6MDY6MDcuNTE2NjgwIiwgIml0ZW1zIjogW3siaXRlbV9pZCI6ICJpdGVtXzY4NiIsICJxdWFudGl0eSI6IDQsICJwcmljZV91c2QiOiA1NC4zfSwgeyJpdGVtX2lkIjogIml0ZW1fNzYyIiwgInF1YW50aXR5IjogNCwgInByaWNlX3VzZCI6IDEzLjY1fSwgeyJpdGVtX2lkIjogIml0ZW1fODI0IiwgInF1YW50aXR5IjogMSwgInByaWNlX3VzZCI6IDY2LjcyfV19""}"
"{""key"": ""event_9"", ""timestamp"": 1753283167.51671, ""value"": ""eyJ1c2VyX2lkIjogInVzZXJfNjcyMyIsICJldmVudF90eXBlIjogInZpZXciLCAiZXZlbnRfdGltZXN0YW1wIjogIjIwMjUtMDctMjNUMTU6MDY6MDcuNTE2Njk2IiwgIml0ZW1zIjogW3siaXRlbV9pZCI6ICJpdGVtXzkxMSIsICJxdWFudGl0eSI6IDIsICJwcmljZV91c2QiOiAyOC4wNX0sIHsiaXRlbV9pZCI6ICJpdGVtXzk4NiIsICJxdWFudGl0eSI6IDQsICJwcmljZV91c2QiOiA5NC43MX0sIHsiaXRlbV9pZCI6ICJpdGVtXzIwNyIsICJxdWFudGl0eSI6IDMsICJwcmljZV91c2QiOiAzMC4zOX1dfQ==""}"


In [0]:
-- create or replace table bronze_lab7_raw
-- as
select *
  ,cast(unbase64(value) as string) as json_str
from read_files(
    DA.paths_working_dir || '/json_demo_files/lab_kafka_events.json'
    , format => 'json'
)

key,timestamp,value,_rescued_data,json_str
event_0,1.753283167516484E9,eyJ1c2VyX2lkIjogInVzZXJfNDQzMCIsICJldmVudF90eXBlIjogInZpZXciLCAiZXZlbnRfdGltZXN0YW1wIjogIjIwMjUtMDctMjNUMTU6MDY6MDcuNTE2NDMyIiwgIml0ZW1zIjogW3siaXRlbV9pZCI6ICJpdGVtXzgxMyIsICJxdWFudGl0eSI6IDEsICJwcmljZV91c2QiOiAzOS4zNn0sIHsiaXRlbV9pZCI6ICJpdGVtXzExMiIsICJxdWFudGl0eSI6IDQsICJwcmljZV91c2QiOiA0MS4xMn1dfQ==,null,"{""user_id"": ""user_4430"", ""event_type"": ""view"", ""event_timestamp"": ""2025-07-23T15:06:07.516432"", ""items"": [{""item_id"": ""item_813"", ""quantity"": 1, ""price_usd"": 39.36}, {""item_id"": ""item_112"", ""quantity"": 4, ""price_usd"": 41.12}]}"
event_1,1.753283167516559E9,eyJ1c2VyX2lkIjogInVzZXJfNTQ4NCIsICJldmVudF90eXBlIjogImNsaWNrIiwgImV2ZW50X3RpbWVzdGFtcCI6ICIyMDI1LTA3LTIzVDE1OjA2OjA3LjUxNjQ5MyIsICJpdGVtcyI6IFt7Iml0ZW1faWQiOiAiaXRlbV82ODgiLCAicXVhbnRpdHkiOiAxLCAicHJpY2VfdXNkIjogMjEuNjh9LCB7Iml0ZW1faWQiOiAiaXRlbV80MjMiLCAicXVhbnRpdHkiOiAyLCAicHJpY2VfdXNkIjogMjIuODJ9LCB7Iml0ZW1faWQiOiAiaXRlbV8yODIiLCAicXVhbnRpdHkiOiAzLCAicHJpY2VfdXNkIjogOTMuNDZ9XX0=,null,"{""user_id"": ""user_5484"", ""event_type"": ""click"", ""event_timestamp"": ""2025-07-23T15:06:07.516493"", ""items"": [{""item_id"": ""item_688"", ""quantity"": 1, ""price_usd"": 21.68}, {""item_id"": ""item_423"", ""quantity"": 2, ""price_usd"": 22.82}, {""item_id"": ""item_282"", ""quantity"": 3, ""price_usd"": 93.46}]}"
event_2,1.753283167516581E9,eyJ1c2VyX2lkIjogInVzZXJfNjkzNSIsICJldmVudF90eXBlIjogInB1cmNoYXNlIiwgImV2ZW50X3RpbWVzdGFtcCI6ICIyMDI1LTA3LTIzVDE1OjA2OjA3LjUxNjU2NCIsICJpdGVtcyI6IFt7Iml0ZW1faWQiOiAiaXRlbV84MjgiLCAicXVhbnRpdHkiOiAyLCAicHJpY2VfdXNkIjogMTQuNTN9XX0=,null,"{""user_id"": ""user_6935"", ""event_type"": ""purchase"", ""event_timestamp"": ""2025-07-23T15:06:07.516564"", ""items"": [{""item_id"": ""item_828"", ""quantity"": 2, ""price_usd"": 14.53}]}"
event_3,1.753283167516599E9,eyJ1c2VyX2lkIjogInVzZXJfMjYyNyIsICJldmVudF90eXBlIjogInB1cmNoYXNlIiwgImV2ZW50X3RpbWVzdGFtcCI6ICIyMDI1LTA3LTIzVDE1OjA2OjA3LjUxNjU4NCIsICJpdGVtcyI6IFt7Iml0ZW1faWQiOiAiaXRlbV85OTkiLCAicXVhbnRpdHkiOiA1LCAicHJpY2VfdXNkIjogOTYuMjN9XX0=,null,"{""user_id"": ""user_2627"", ""event_type"": ""purchase"", ""event_timestamp"": ""2025-07-23T15:06:07.516584"", ""items"": [{""item_id"": ""item_999"", ""quantity"": 5, ""price_usd"": 96.23}]}"
event_4,1.753283167516619E9,eyJ1c2VyX2lkIjogInVzZXJfNDQ0NiIsICJldmVudF90eXBlIjogInZpZXciLCAiZXZlbnRfdGltZXN0YW1wIjogIjIwMjUtMDctMjNUMTU6MDY6MDcuNTE2NjAzIiwgIml0ZW1zIjogW3siaXRlbV9pZCI6ICJpdGVtXzQ4NyIsICJxdWFudGl0eSI6IDUsICJwcmljZV91c2QiOiA1MC45OX1dfQ==,null,"{""user_id"": ""user_4446"", ""event_type"": ""view"", ""event_timestamp"": ""2025-07-23T15:06:07.516603"", ""items"": [{""item_id"": ""item_487"", ""quantity"": 5, ""price_usd"": 50.99}]}"
event_5,1.753283167516643E9,eyJ1c2VyX2lkIjogInVzZXJfMzM0NSIsICJldmVudF90eXBlIjogInZpZXciLCAiZXZlbnRfdGltZXN0YW1wIjogIjIwMjUtMDctMjNUMTU6MDY6MDcuNTE2NjIzIiwgIml0ZW1zIjogW3siaXRlbV9pZCI6ICJpdGVtXzY0OSIsICJxdWFudGl0eSI6IDEsICJwcmljZV91c2QiOiAxNi4xM30sIHsiaXRlbV9pZCI6ICJpdGVtXzY4NCIsICJxdWFudGl0eSI6IDQsICJwcmljZV91c2QiOiA0OC43Nn0sIHsiaXRlbV9pZCI6ICJpdGVtXzYyNSIsICJxdWFudGl0eSI6IDQsICJwcmljZV91c2QiOiA4NC4wNH1dfQ==,null,"{""user_id"": ""user_3345"", ""event_type"": ""view"", ""event_timestamp"": ""2025-07-23T15:06:07.516623"", ""items"": [{""item_id"": ""item_649"", ""quantity"": 1, ""price_usd"": 16.13}, {""item_id"": ""item_684"", ""quantity"": 4, ""price_usd"": 48.76}, {""item_id"": ""item_625"", ""quantity"": 4, ""price_usd"": 84.04}]}"
event_6,1.753283167516666E9,eyJ1c2VyX2lkIjogInVzZXJfOTY0MSIsICJldmVudF90eXBlIjogInZpZXciLCAiZXZlbnRfdGltZXN0YW1wIjogIjIwMjUtMDctMjNUMTU6MDY6MDcuNTE2NjQ1IiwgIml0ZW1zIjogW3siaXRlbV9pZCI6ICJpdGVtXzk4MCIsICJxdWFudGl0eSI6IDQsICJwcmljZV91c2QiOiA4MC4wMn0sIHsiaXRlbV9pZCI6ICJpdGVtXzc2OSIsICJxdWFudGl0eSI6IDMsICJwcmljZV91c2QiOiAzMy4zNn0sIHsiaXRlbV9pZCI6ICJpdGVtXzU1MCIsICJxdWFudGl0eSI6IDQsICJwcmljZV91c2QiOiAzNy4zN31dfQ==,null,"{""user_id"": ""user_9641"", ""event_type"": ""view"", ""event_timestamp"": ""2025-07-2

In [0]:
CREATE OR REPLACE TABLE lab7_lab_kafka_events_raw
AS
SELECT 
  *,
  cast(unbase64(value) as STRING) as decoded_value
FROM read_files(
        DA.paths_working_dir || '/json_demo_files/lab_kafka_events.json',
        format => "json", 
        schema => '''
          key STRING, 
          timestamp DOUBLE, 
          value STRING
        ''',
        rescueddatacolumn => '_rescued_data'
      );

-- View the table
SELECT *
FROM lab7_lab_kafka_events_raw;

key,timestamp,value,_rescued_data,decoded_value
event_0,1.753283167516484E9,eyJ1c2VyX2lkIjogInVzZXJfNDQzMCIsICJldmVudF90eXBlIjogInZpZXciLCAiZXZlbnRfdGltZXN0YW1wIjogIjIwMjUtMDctMjNUMTU6MDY6MDcuNTE2NDMyIiwgIml0ZW1zIjogW3siaXRlbV9pZCI6ICJpdGVtXzgxMyIsICJxdWFudGl0eSI6IDEsICJwcmljZV91c2QiOiAzOS4zNn0sIHsiaXRlbV9pZCI6ICJpdGVtXzExMiIsICJxdWFudGl0eSI6IDQsICJwcmljZV91c2QiOiA0MS4xMn1dfQ==,null,"{""user_id"": ""user_4430"", ""event_type"": ""view"", ""event_timestamp"": ""2025-07-23T15:06:07.516432"", ""items"": [{""item_id"": ""item_813"", ""quantity"": 1, ""price_usd"": 39.36}, {""item_id"": ""item_112"", ""quantity"": 4, ""price_usd"": 41.12}]}"
event_1,1.753283167516559E9,eyJ1c2VyX2lkIjogInVzZXJfNTQ4NCIsICJldmVudF90eXBlIjogImNsaWNrIiwgImV2ZW50X3RpbWVzdGFtcCI6ICIyMDI1LTA3LTIzVDE1OjA2OjA3LjUxNjQ5MyIsICJpdGVtcyI6IFt7Iml0ZW1faWQiOiAiaXRlbV82ODgiLCAicXVhbnRpdHkiOiAxLCAicHJpY2VfdXNkIjogMjEuNjh9LCB7Iml0ZW1faWQiOiAiaXRlbV80MjMiLCAicXVhbnRpdHkiOiAyLCAicHJpY2VfdXNkIjogMjIuODJ9LCB7Iml0ZW1faWQiOiAiaXRlbV8yODIiLCAicXVhbnRpdHkiOiAzLCAicHJpY2VfdXNkIjogOTMuNDZ9XX0=,null,"{""user_id"": ""user_5484"", ""event_type"": ""click"", ""event_timestamp"": ""2025-07-23T15:06:07.516493"", ""items"": [{""item_id"": ""item_688"", ""quantity"": 1, ""price_usd"": 21.68}, {""item_id"": ""item_423"", ""quantity"": 2, ""price_usd"": 22.82}, {""item_id"": ""item_282"", ""quantity"": 3, ""price_usd"": 93.46}]}"
event_2,1.753283167516581E9,eyJ1c2VyX2lkIjogInVzZXJfNjkzNSIsICJldmVudF90eXBlIjogInB1cmNoYXNlIiwgImV2ZW50X3RpbWVzdGFtcCI6ICIyMDI1LTA3LTIzVDE1OjA2OjA3LjUxNjU2NCIsICJpdGVtcyI6IFt7Iml0ZW1faWQiOiAiaXRlbV84MjgiLCAicXVhbnRpdHkiOiAyLCAicHJpY2VfdXNkIjogMTQuNTN9XX0=,null,"{""user_id"": ""user_6935"", ""event_type"": ""purchase"", ""event_timestamp"": ""2025-07-23T15:06:07.516564"", ""items"": [{""item_id"": ""item_828"", ""quantity"": 2, ""price_usd"": 14.53}]}"
event_3,1.753283167516599E9,eyJ1c2VyX2lkIjogInVzZXJfMjYyNyIsICJldmVudF90eXBlIjogInB1cmNoYXNlIiwgImV2ZW50X3RpbWVzdGFtcCI6ICIyMDI1LTA3LTIzVDE1OjA2OjA3LjUxNjU4NCIsICJpdGVtcyI6IFt7Iml0ZW1faWQiOiAiaXRlbV85OTkiLCAicXVhbnRpdHkiOiA1LCAicHJpY2VfdXNkIjogOTYuMjN9XX0=,null,"{""user_id"": ""user_2627"", ""event_type"": ""purchase"", ""event_timestamp"": ""2025-07-23T15:06:07.516584"", ""items"": [{""item_id"": ""item_999"", ""quantity"": 5, ""price_usd"": 96.23}]}"
event_4,1.753283167516619E9,eyJ1c2VyX2lkIjogInVzZXJfNDQ0NiIsICJldmVudF90eXBlIjogInZpZXciLCAiZXZlbnRfdGltZXN0YW1wIjogIjIwMjUtMDctMjNUMTU6MDY6MDcuNTE2NjAzIiwgIml0ZW1zIjogW3siaXRlbV9pZCI6ICJpdGVtXzQ4NyIsICJxdWFudGl0eSI6IDUsICJwcmljZV91c2QiOiA1MC45OX1dfQ==,null,"{""user_id"": ""user_4446"", ""event_type"": ""view"", ""event_timestamp"": ""2025-07-23T15:06:07.516603"", ""items"": [{""item_id"": ""item_487"", ""quantity"": 5, ""price_usd"": 50.99}]}"
event_5,1.753283167516643E9,eyJ1c2VyX2lkIjogInVzZXJfMzM0NSIsICJldmVudF90eXBlIjogInZpZXciLCAiZXZlbnRfdGltZXN0YW1wIjogIjIwMjUtMDctMjNUMTU6MDY6MDcuNTE2NjIzIiwgIml0ZW1zIjogW3siaXRlbV9pZCI6ICJpdGVtXzY0OSIsICJxdWFudGl0eSI6IDEsICJwcmljZV91c2QiOiAxNi4xM30sIHsiaXRlbV9pZCI6ICJpdGVtXzY4NCIsICJxdWFudGl0eSI6IDQsICJwcmljZV91c2QiOiA0OC43Nn0sIHsiaXRlbV9pZCI6ICJpdGVtXzYyNSIsICJxdWFudGl0eSI6IDQsICJwcmljZV91c2QiOiA4NC4wNH1dfQ==,null,"{""user_id"": ""user_3345"", ""event_type"": ""view"", ""event_timestamp"": ""2025-07-23T15:06:07.516623"", ""items"": [{""item_id"": ""item_649"", ""quantity"": 1, ""price_usd"": 16.13}, {""item_id"": ""item_684"", ""quantity"": 4, ""price_usd"": 48.76}, {""item_id"": ""item_625"", ""quantity"": 4, ""price_usd"": 84.04}]}"
event_6,1.753283167516666E9,eyJ1c2VyX2lkIjogInVzZXJfOTY0MSIsICJldmVudF90eXBlIjogInZpZXciLCAiZXZlbnRfdGltZXN0YW1wIjogIjIwMjUtMDctMjNUMTU6MDY6MDcuNTE2NjQ1IiwgIml0ZW1zIjogW3siaXRlbV9pZCI6ICJpdGVtXzk4MCIsICJxdWFudGl0eSI6IDQsICJwcmljZV91c2QiOiA4MC4wMn0sIHsiaXRlbV9pZCI6ICJpdGVtXzc2OSIsICJxdWFudGl0eSI6IDMsICJwcmljZV91c2QiOiAzMy4zNn0sIHsiaXRlbV9pZCI6ICJpdGVtXzU1MCIsICJxdWFudGl0eSI6IDQsICJwcmljZV91c2QiOiAzNy4zN31dfQ==,null,"{""user_id"": ""user_9641"", ""event_type"": ""view"", ""event_timestamp"": ""2025

### B3. Create the Flattened Bronze Table

1. Your goal is to flatten the JSON formatted string column **decoded_value** from the table **lab7_lab_kafka_events_raw** to create a new table named **lab7_lab_kafka_events_flattened** for downstream processing. The table should contain the following columns:
    - **key**
    - **timestamp**
    - **user_id**
    - **event_type**
    - **event_timestamp**
    - **items**

    You can use whichever technique you prefer:

    - Parse the JSON formatted string (easiest) to flatten
      - [Query JSON strings](https://docs.databricks.com/aws/en/semi-structured/json):

    - Convert the JSON formatted string as a VARIANT and flatten
      - [parse_json function](https://docs.databricks.com/gcp/en/sql/language-manual/functions/parse_json)

    - Convert the JSON formatted string to a STRUCT and flatten
      - [schema_of_json function](https://docs.databricks.com/aws/en/sql/language-manual/functions/schema_of_json)
      - [from_json function](https://docs.databricks.com/gcp/en/sql/language-manual/functions/from_json)

**NOTE:** View the lab solution notebook to view the solutions for each.

1. To begin, run the code below to view the final solution table **lab7_lab_kafka_events_flattened_solution**. This will give you an idea of what your final table should look like.

  **NOTE**: Depending on your solution, the data types of the columns may vary slightly.  


##### Optional Challenge

  As a challenge, after flattening the table, try converting the data types accordingly. Depending on your skill set, you may not convert all columns to the correct data types within the allotted time.

  - **key** STRING
  - **timestamp** DOUBLE
  - **user_id** STRING
  - **event_type** STRING
  - **event_timestamp** TIMESTAMP
  - **items** (STRUCT or VARIANT) depending on the method you used.

In [0]:
SELECT *
FROM lab7_lab_kafka_events_flattened_solution

key,timestamp,user_id,event_type,event_timestamp,items
event_0,1.753283167516484E9,user_4430,view,2025-07-23T15:06:07.516432Z,"List(List(item_813, 39.36, 1), List(item_112, 41.12, 4))"
event_1,1.753283167516559E9,user_5484,click,2025-07-23T15:06:07.516493Z,"List(List(item_688, 21.68, 1), List(item_423, 22.82, 2), List(item_282, 93.46, 3))"
event_2,1.753283167516581E9,user_6935,purchase,2025-07-23T15:06:07.516564Z,"List(List(item_828, 14.53, 2))"
event_3,1.753283167516599E9,user_2627,purchase,2025-07-23T15:06:07.516584Z,"List(List(item_999, 96.23, 5))"
event_4,1.753283167516619E9,user_4446,view,2025-07-23T15:06:07.516603Z,"List(List(item_487, 50.99, 5))"
event_5,1.753283167516643E9,user_3345,view,2025-07-23T15:06:07.516623Z,"List(List(item_649, 16.13, 1), List(item_684, 48.76, 4), List(item_625, 84.04, 4))"
event_6,1.753283167516666E9,user_9641,view,2025-07-23T15:06:07.516645Z,"List(List(item_980, 80.02, 4), List(item_769, 33.36, 3), List(item_550, 37.37, 4))"
event_7,1.753283167516678E9,user_6128,click,2025-07-23T15:06:07.516668Z,"List(List(item_243, 37.02, 5))"
event_8,1.753283167516694E9,user_8096,view,2025-07-23T15:06:07.51668Z,"List(List(item_686, 54.3, 4), List(item_762, 13.65, 4), List(item_824, 66.72, 1))"
event_9,1.75328316751671E9,user_6723,view,2025-07-23T15:06:07.516696Z,"List(List(item_911, 28.05, 2), List(item_986, 94.71, 4), List(item_207, 30.39, 3))"


2. Write the query in the cell below to read the **lab_kafka_events_raw** table and create the flattened table **lab7_lab_kafka_events_flattened** following the requirements from above.

In [0]:
create or replace table bronze_lab7_raw
select *
  ,cast(unbase64(value) as string) as decoded_value
from read_files(
    DA.paths_working_dir || '/json_demo_files/lab_kafka_events.json'
    , format => 'json'
    , schema => '''
    key STRING
    , timestamp DOUBLE
    , value STRING
    '''
)

num_affected_rows,num_inserted_rows


In [0]:
select schema_of_json(decoded_value) as j_schema
  ,count(1)
from bronze_lab7_raw
group by schema_of_json(decoded_value)
order by 1

j_schema,count(1)
"STRUCT>, user_id: STRING>",100


In [0]:
create or replace table bronze_lab7_parsed
select *
  ,from_unixtime(timestamp) as record_date
  ,from_json(decoded_value, 'STRUCT<event_timestamp: STRING, event_type: STRING, items: ARRAY<STRUCT<item_id: STRING, price_usd: DOUBLE, quantity: BIGINT>>, user_id: STRING>') as json_struct
  ,parse_json(decoded_value) as json_variant
from bronze_lab7_raw;

select *
from bronze_lab7_parsed

key,timestamp,value,decoded_value,record_date,json_struct,json_variant
event_0,1.753283167516484E9,eyJ1c2VyX2lkIjogInVzZXJfNDQzMCIsICJldmVudF90eXBlIjogInZpZXciLCAiZXZlbnRfdGltZXN0YW1wIjogIjIwMjUtMDctMjNUMTU6MDY6MDcuNTE2NDMyIiwgIml0ZW1zIjogW3siaXRlbV9pZCI6ICJpdGVtXzgxMyIsICJxdWFudGl0eSI6IDEsICJwcmljZV91c2QiOiAzOS4zNn0sIHsiaXRlbV9pZCI6ICJpdGVtXzExMiIsICJxdWFudGl0eSI6IDQsICJwcmljZV91c2QiOiA0MS4xMn1dfQ==,"{""user_id"": ""user_4430"", ""event_type"": ""view"", ""event_timestamp"": ""2025-07-23T15:06:07.516432"", ""items"": [{""item_id"": ""item_813"", ""quantity"": 1, ""price_usd"": 39.36}, {""item_id"": ""item_112"", ""quantity"": 4, ""price_usd"": 41.12}]}",2025-07-23 15:06:07,"List(2025-07-23T15:06:07.516432, view, List(List(item_813, 39.36, 1), List(item_112, 41.12, 4)), user_4430)","{""event_timestamp"":""2025-07-23T15:06:07.516432"",""event_type"":""view"",""items"":[{""item_id"":""item_813"",""price_usd"":39.36,""quantity"":1},{""item_id"":""item_112"",""price_usd"":41.12,""quantity"":4}],""user_id"":""user_4430""}"
event_1,1.753283167516559E9,eyJ1c2VyX2lkIjogInVzZXJfNTQ4NCIsICJldmVudF90eXBlIjogImNsaWNrIiwgImV2ZW50X3RpbWVzdGFtcCI6ICIyMDI1LTA3LTIzVDE1OjA2OjA3LjUxNjQ5MyIsICJpdGVtcyI6IFt7Iml0ZW1faWQiOiAiaXRlbV82ODgiLCAicXVhbnRpdHkiOiAxLCAicHJpY2VfdXNkIjogMjEuNjh9LCB7Iml0ZW1faWQiOiAiaXRlbV80MjMiLCAicXVhbnRpdHkiOiAyLCAicHJpY2VfdXNkIjogMjIuODJ9LCB7Iml0ZW1faWQiOiAiaXRlbV8yODIiLCAicXVhbnRpdHkiOiAzLCAicHJpY2VfdXNkIjogOTMuNDZ9XX0=,"{""user_id"": ""user_5484"", ""event_type"": ""click"", ""event_timestamp"": ""2025-07-23T15:06:07.516493"", ""items"": [{""item_id"": ""item_688"", ""quantity"": 1, ""price_usd"": 21.68}, {""item_id"": ""item_423"", ""quantity"": 2, ""price_usd"": 22.82}, {""item_id"": ""item_282"", ""quantity"": 3, ""price_usd"": 93.46}]}",2025-07-23 15:06:07,"List(2025-07-23T15:06:07.516493, click, List(List(item_688, 21.68, 1), List(item_423, 22.82, 2), List(item_282, 93.46, 3)), user_5484)","{""event_timestamp"":""2025-07-23T15:06:07.516493"",""event_type"":""click"",""items"":[{""item_id"":""item_688"",""price_usd"":21.68,""quantity"":1},{""item_id"":""item_423"",""price_usd"":22.82,""quantity"":2},{""item_id"":""item_282"",""price_usd"":93.46,""quantity"":3}],""user_id"":""user_5484""}"
event_2,1.753283167516581E9,eyJ1c2VyX2lkIjogInVzZXJfNjkzNSIsICJldmVudF90eXBlIjogInB1cmNoYXNlIiwgImV2ZW50X3RpbWVzdGFtcCI6ICIyMDI1LTA3LTIzVDE1OjA2OjA3LjUxNjU2NCIsICJpdGVtcyI6IFt7Iml0ZW1faWQiOiAiaXRlbV84MjgiLCAicXVhbnRpdHkiOiAyLCAicHJpY2VfdXNkIjogMTQuNTN9XX0=,"{""user_id"": ""user_6935"", ""event_type"": ""purchase"", ""event_timestamp"": ""2025-07-23T15:06:07.516564"", ""items"": [{""item_id"": ""item_828"", ""quantity"": 2, ""price_usd"": 14.53}]}",2025-07-23 15:06:07,"List(2025-07-23T15:06:07.516564, purchase, List(List(item_828, 14.53, 2)), user_6935)","{""event_timestamp"":""2025-07-23T15:06:07.516564"",""event_type"":""purchase"",""items"":[{""item_id"":""item_828"",""price_usd"":14.53,""quantity"":2}],""user_id"":""user_6935""}"
event_3,1.753283167516599E9,eyJ1c2VyX2lkIjogInVzZXJfMjYyNyIsICJldmVudF90eXBlIjogInB1cmNoYXNlIiwgImV2ZW50X3RpbWVzdGFtcCI6ICIyMDI1LTA3LTIzVDE1OjA2OjA3LjUxNjU4NCIsICJpdGVtcyI6IFt7Iml0ZW1faWQiOiAiaXRlbV85OTkiLCAicXVhbnRpdHkiOiA1LCAicHJpY2VfdXNkIjogOTYuMjN9XX0=,"{""user_id"": ""user_2627"", ""event_type"": ""purchase"", ""event_timestamp"": ""2025-07-23T15:06:07.516584"", ""items"": [{""item_id"": ""item_999"", ""quantity"": 5, ""price_usd"": 96.23}]}",2025-07-23 15:06:07,"List(2025-07-23T15:06:07.516584, purchase, List(List(item_999, 96.23, 5)), user_2627)","{""event_timestamp"":""2025-07-23T15:06:07.516584"",""event_type"":""purchase"",""items"":[{""item_id"":""item_999"",""price_usd"":96.23,""quantity"":5}],""user_id"":""user_2627""}"
event_4,1.753283167516619E9,eyJ1c2VyX2lkIjogInVzZXJfNDQ0NiIsICJldmVudF90eXBlIjogInZpZXciLCAiZXZlbnRfdGltZXN0YW1wIjogIjIwMjUtMDctMjNUMTU6MDY6MDcuNTE2NjAzIiwgIml0ZW1zIjogW3siaXRlbV9pZCI6ICJpdGVtXzQ4NyIsICJxdWFudGl0eSI6IDUsICJwcmljZV91c2QiOiA1MC45OX1dfQ==,"{""user_id"": ""user_444

In [0]:
select *
  ,json_struct.user_id
  ,json_struct.event_type
  ,json_struct.event_timestamp
  ,json_struct.items
from bronze_lab7_parsed

key,timestamp,value,decoded_value,record_date,json_struct,json_variant,user_id,event_timestamp,items
event_0,1.753283167516484E9,eyJ1c2VyX2lkIjogInVzZXJfNDQzMCIsICJldmVudF90eXBlIjogInZpZXciLCAiZXZlbnRfdGltZXN0YW1wIjogIjIwMjUtMDctMjNUMTU6MDY6MDcuNTE2NDMyIiwgIml0ZW1zIjogW3siaXRlbV9pZCI6ICJpdGVtXzgxMyIsICJxdWFudGl0eSI6IDEsICJwcmljZV91c2QiOiAzOS4zNn0sIHsiaXRlbV9pZCI6ICJpdGVtXzExMiIsICJxdWFudGl0eSI6IDQsICJwcmljZV91c2QiOiA0MS4xMn1dfQ==,"{""user_id"": ""user_4430"", ""event_type"": ""view"", ""event_timestamp"": ""2025-07-23T15:06:07.516432"", ""items"": [{""item_id"": ""item_813"", ""quantity"": 1, ""price_usd"": 39.36}, {""item_id"": ""item_112"", ""quantity"": 4, ""price_usd"": 41.12}]}",2025-07-23 15:06:07,"List(2025-07-23T15:06:07.516432, view, List(List(item_813, 39.36, 1), List(item_112, 41.12, 4)), user_4430)","{""event_timestamp"":""2025-07-23T15:06:07.516432"",""event_type"":""view"",""items"":[{""item_id"":""item_813"",""price_usd"":39.36,""quantity"":1},{""item_id"":""item_112"",""price_usd"":41.12,""quantity"":4}],""user_id"":""user_4430""}",user_4430,2025-07-23T15:06:07.516432,"List(List(item_813, 39.36, 1), List(item_112, 41.12, 4))"
event_1,1.753283167516559E9,eyJ1c2VyX2lkIjogInVzZXJfNTQ4NCIsICJldmVudF90eXBlIjogImNsaWNrIiwgImV2ZW50X3RpbWVzdGFtcCI6ICIyMDI1LTA3LTIzVDE1OjA2OjA3LjUxNjQ5MyIsICJpdGVtcyI6IFt7Iml0ZW1faWQiOiAiaXRlbV82ODgiLCAicXVhbnRpdHkiOiAxLCAicHJpY2VfdXNkIjogMjEuNjh9LCB7Iml0ZW1faWQiOiAiaXRlbV80MjMiLCAicXVhbnRpdHkiOiAyLCAicHJpY2VfdXNkIjogMjIuODJ9LCB7Iml0ZW1faWQiOiAiaXRlbV8yODIiLCAicXVhbnRpdHkiOiAzLCAicHJpY2VfdXNkIjogOTMuNDZ9XX0=,"{""user_id"": ""user_5484"", ""event_type"": ""click"", ""event_timestamp"": ""2025-07-23T15:06:07.516493"", ""items"": [{""item_id"": ""item_688"", ""quantity"": 1, ""price_usd"": 21.68}, {""item_id"": ""item_423"", ""quantity"": 2, ""price_usd"": 22.82}, {""item_id"": ""item_282"", ""quantity"": 3, ""price_usd"": 93.46}]}",2025-07-23 15:06:07,"List(2025-07-23T15:06:07.516493, click, List(List(item_688, 21.68, 1), List(item_423, 22.82, 2), List(item_282, 93.46, 3)), user_5484)","{""event_timestamp"":""2025-07-23T15:06:07.516493"",""event_type"":""click"",""items"":[{""item_id"":""item_688"",""price_usd"":21.68,""quantity"":1},{""item_id"":""item_423"",""price_usd"":22.82,""quantity"":2},{""item_id"":""item_282"",""price_usd"":93.46,""quantity"":3}],""user_id"":""user_5484""}",user_5484,2025-07-23T15:06:07.516493,"List(List(item_688, 21.68, 1), List(item_423, 22.82, 2), List(item_282, 93.46, 3))"
event_2,1.753283167516581E9,eyJ1c2VyX2lkIjogInVzZXJfNjkzNSIsICJldmVudF90eXBlIjogInB1cmNoYXNlIiwgImV2ZW50X3RpbWVzdGFtcCI6ICIyMDI1LTA3LTIzVDE1OjA2OjA3LjUxNjU2NCIsICJpdGVtcyI6IFt7Iml0ZW1faWQiOiAiaXRlbV84MjgiLCAicXVhbnRpdHkiOiAyLCAicHJpY2VfdXNkIjogMTQuNTN9XX0=,"{""user_id"": ""user_6935"", ""event_type"": ""purchase"", ""event_timestamp"": ""2025-07-23T15:06:07.516564"", ""items"": [{""item_id"": ""item_828"", ""quantity"": 2, ""price_usd"": 14.53}]}",2025-07-23 15:06:07,"List(2025-07-23T15:06:07.516564, purchase, List(List(item_828, 14.53, 2)), user_6935)","{""event_timestamp"":""2025-07-23T15:06:07.516564"",""event_type"":""purchase"",""items"":[{""item_id"":""item_828"",""price_usd"":14.53,""quantity"":2}],""user_id"":""user_6935""}",user_6935,2025-07-23T15:06:07.516564,"List(List(item_828, 14.53, 2))"
event_3,1.753283167516599E9,eyJ1c2VyX2lkIjogInVzZXJfMjYyNyIsICJldmVudF90eXBlIjogInB1cmNoYXNlIiwgImV2ZW50X3RpbWVzdGFtcCI6ICIyMDI1LTA3LTIzVDE1OjA2OjA3LjUxNjU4NCIsICJpdGVtcyI6IFt7Iml0ZW1faWQiOiAiaXRlbV85OTkiLCAicXVhbnRpdHkiOiA1LCAicHJpY2VfdXNkIjogOTYuMjN9XX0=,"{""user_id"": ""user_2627"", ""event_type"": ""purchase"", ""event_timestamp"": ""2025-07-23T15:06:07.516584"", ""items"": [{""item_id"": ""item_999"", ""quantity"": 5, ""price_usd"": 96.23}]}",2025-07-23 15:06:07,"List(2025-07-23T15:06:07.516584, purchase, List(List(item_999, 96.23, 5)), user_2627)","{""event_timestamp"":""2025-07-23T15:06:07.516584"",""event_type"":""purchase"",""items"":[{""item_id"":""item_999"",""price_usd"":96.23,""qua

In [0]:
select key, timestamp, record_date
  ,json_variant:user_id :: string
  ,json_variant:event_type :: string
  ,json_variant:event_timestamp :: timestamp
  ,json_variant:items :: array<struct<item_id string, price_usd double, quantity int>>
from bronze_lab7_parsed

key,timestamp,record_date,user_id,event_type,event_timestamp,items
event_0,1.753283167516484E9,2025-07-23 15:06:07,user_4430,view,2025-07-23T15:06:07.516432Z,"List(List(item_813, 39.36, 1), List(item_112, 41.12, 4))"
event_1,1.753283167516559E9,2025-07-23 15:06:07,user_5484,click,2025-07-23T15:06:07.516493Z,"List(List(item_688, 21.68, 1), List(item_423, 22.82, 2), List(item_282, 93.46, 3))"
event_2,1.753283167516581E9,2025-07-23 15:06:07,user_6935,purchase,2025-07-23T15:06:07.516564Z,"List(List(item_828, 14.53, 2))"
event_3,1.753283167516599E9,2025-07-23 15:06:07,user_2627,purchase,2025-07-23T15:06:07.516584Z,"List(List(item_999, 96.23, 5))"
event_4,1.753283167516619E9,2025-07-23 15:06:07,user_4446,view,2025-07-23T15:06:07.516603Z,"List(List(item_487, 50.99, 5))"
event_5,1.753283167516643E9,2025-07-23 15:06:07,user_3345,view,2025-07-23T15:06:07.516623Z,"List(List(item_649, 16.13, 1), List(item_684, 48.76, 4), List(item_625, 84.04, 4))"
event_6,1.753283167516666E9,2025-07-23 15:06:07,user_9641,view,2025-07-23T15:06:07.516645Z,"List(List(item_980, 80.02, 4), List(item_769, 33.36, 3), List(item_550, 37.37, 4))"
event_7,1.753283167516678E9,2025-07-23 15:06:07,user_6128,click,2025-07-23T15:06:07.516668Z,"List(List(item_243, 37.02, 5))"
event_8,1.753283167516694E9,2025-07-23 15:06:07,user_8096,view,2025-07-23T15:06:07.51668Z,"List(List(item_686, 54.3, 4), List(item_762, 13.65, 4), List(item_824, 66.72, 1))"
event_9,1.75328316751671E9,2025-07-23 15:06:07,user_6723,view,2025-07-23T15:06:07.516696Z,"List(List(item_911, 28.05, 2), List(item_986, 94.71, 4), List(item_207, 30.39, 3))"


In [0]:
select *
  ,json_struct.user_id
  ,json_struct.event_type
  ,json_struct.event_timestamp
  ,json_struct.items
  ,item.*
from bronze_lab7_parsed
LATERAL VIEW explode(json_struct.items) AS item

key,timestamp,value,decoded_value,record_date,json_struct,json_variant,item,user_id,event_type,event_timestamp,items,item_id,price_usd,quantity
event_0,1.753283167516484E9,eyJ1c2VyX2lkIjogInVzZXJfNDQzMCIsICJldmVudF90eXBlIjogInZpZXciLCAiZXZlbnRfdGltZXN0YW1wIjogIjIwMjUtMDctMjNUMTU6MDY6MDcuNTE2NDMyIiwgIml0ZW1zIjogW3siaXRlbV9pZCI6ICJpdGVtXzgxMyIsICJxdWFudGl0eSI6IDEsICJwcmljZV91c2QiOiAzOS4zNn0sIHsiaXRlbV9pZCI6ICJpdGVtXzExMiIsICJxdWFudGl0eSI6IDQsICJwcmljZV91c2QiOiA0MS4xMn1dfQ==,"{""user_id"": ""user_4430"", ""event_type"": ""view"", ""event_timestamp"": ""2025-07-23T15:06:07.516432"", ""items"": [{""item_id"": ""item_813"", ""quantity"": 1, ""price_usd"": 39.36}, {""item_id"": ""item_112"", ""quantity"": 4, ""price_usd"": 41.12}]}",2025-07-23 15:06:07,"List(2025-07-23T15:06:07.516432, view, List(List(item_813, 39.36, 1), List(item_112, 41.12, 4)), user_4430)","{""event_timestamp"":""2025-07-23T15:06:07.516432"",""event_type"":""view"",""items"":[{""item_id"":""item_813"",""price_usd"":39.36,""quantity"":1},{""item_id"":""item_112"",""price_usd"":41.12,""quantity"":4}],""user_id"":""user_4430""}","List(item_813, 39.36, 1)",user_4430,view,2025-07-23T15:06:07.516432,"List(List(item_813, 39.36, 1), List(item_112, 41.12, 4))",item_813,39.36,1
event_0,1.753283167516484E9,eyJ1c2VyX2lkIjogInVzZXJfNDQzMCIsICJldmVudF90eXBlIjogInZpZXciLCAiZXZlbnRfdGltZXN0YW1wIjogIjIwMjUtMDctMjNUMTU6MDY6MDcuNTE2NDMyIiwgIml0ZW1zIjogW3siaXRlbV9pZCI6ICJpdGVtXzgxMyIsICJxdWFudGl0eSI6IDEsICJwcmljZV91c2QiOiAzOS4zNn0sIHsiaXRlbV9pZCI6ICJpdGVtXzExMiIsICJxdWFudGl0eSI6IDQsICJwcmljZV91c2QiOiA0MS4xMn1dfQ==,"{""user_id"": ""user_4430"", ""event_type"": ""view"", ""event_timestamp"": ""2025-07-23T15:06:07.516432"", ""items"": [{""item_id"": ""item_813"", ""quantity"": 1, ""price_usd"": 39.36}, {""item_id"": ""item_112"", ""quantity"": 4, ""price_usd"": 41.12}]}",2025-07-23 15:06:07,"List(2025-07-23T15:06:07.516432, view, List(List(item_813, 39.36, 1), List(item_112, 41.12, 4)), user_4430)","{""event_timestamp"":""2025-07-23T15:06:07.516432"",""event_type"":""view"",""items"":[{""item_id"":""item_813"",""price_usd"":39.36,""quantity"":1},{""item_id"":""item_112"",""price_usd"":41.12,""quantity"":4}],""user_id"":""user_4430""}","List(item_112, 41.12, 4)",user_4430,view,2025-07-23T15:06:07.516432,"List(List(item_813, 39.36, 1), List(item_112, 41.12, 4))",item_112,41.12,4
event_1,1.753283167516559E9,eyJ1c2VyX2lkIjogInVzZXJfNTQ4NCIsICJldmVudF90eXBlIjogImNsaWNrIiwgImV2ZW50X3RpbWVzdGFtcCI6ICIyMDI1LTA3LTIzVDE1OjA2OjA3LjUxNjQ5MyIsICJpdGVtcyI6IFt7Iml0ZW1faWQiOiAiaXRlbV82ODgiLCAicXVhbnRpdHkiOiAxLCAicHJpY2VfdXNkIjogMjEuNjh9LCB7Iml0ZW1faWQiOiAiaXRlbV80MjMiLCAicXVhbnRpdHkiOiAyLCAicHJpY2VfdXNkIjogMjIuODJ9LCB7Iml0ZW1faWQiOiAiaXRlbV8yODIiLCAicXVhbnRpdHkiOiAzLCAicHJpY2VfdXNkIjogOTMuNDZ9XX0=,"{""user_id"": ""user_5484"", ""event_type"": ""click"", ""event_timestamp"": ""2025-07-23T15:06:07.516493"", ""items"": [{""item_id"": ""item_688"", ""quantity"": 1, ""price_usd"": 21.68}, {""item_id"": ""item_423"", ""quantity"": 2, ""price_usd"": 22.82}, {""item_id"": ""item_282"", ""quantity"": 3, ""price_usd"": 93.46}]}",2025-07-23 15:06:07,"List(2025-07-23T15:06:07.516493, click, List(List(item_688, 21.68, 1), List(item_423, 22.82, 2), List(item_282, 93.46, 3)), user_5484)","{""event_timestamp"":""2025-07-23T15:06:07.516493"",""event_type"":""click"",""items"":[{""item_id"":""item_688"",""price_usd"":21.68,""quantity"":1},{""item_id"":""item_423"",""price_usd"":22.82,""quantity"":2},{""item_id"":""item_282"",""price_usd"":93.46,""quantity"":3}],""user_id"":""user_5484""}","List(item_688, 21.68, 1)",user_5484,click,2025-07-23T15:06:07.516493,"List(List(item_688, 21.68, 1), List(item_423, 22.82, 2), List(item_282, 93.46, 3))",item_688,21.68,1
event_1,1.753283167516559E9,eyJ1c2VyX2lkIjogInVzZXJfNTQ4NCIsICJldmVudF90eXBlIjogImNsaWNrIiwgImV2ZW50X3RpbWVzdGFtcCI6ICIyMDI1LTA3LTIzVDE1OjA2OjA3LjUxNjQ5MyIsICJpdGVtcyI6IFt7Iml0ZW1faWQiOiAiaXRlbV82ODgiLCAicXVhbnRpdHkiOiAxLCAicHJpY2VfdXNkIjogMjEuNjh9LCB7Iml0ZW1faWQiOi

In [0]:
with test as (
  select key, timestamp, record_date
    ,json_variant:user_id :: string
    ,json_variant:event_type :: string
    ,json_variant:event_timestamp :: timestamp
    ,json_variant:items :: array<struct<item_id string, price_usd double, quantity int>>
  from bronze_lab7_parsed
)
select *
  , item.*
from test
lateral view explode(items) i as item

key,timestamp,record_date,user_id,event_type,event_timestamp,items,item,item_id,price_usd,quantity
event_0,1.753283167516484E9,2025-07-23 15:06:07,user_4430,view,2025-07-23T15:06:07.516432Z,"List(List(item_813, 39.36, 1), List(item_112, 41.12, 4))","List(item_813, 39.36, 1)",item_813,39.36,1
event_0,1.753283167516484E9,2025-07-23 15:06:07,user_4430,view,2025-07-23T15:06:07.516432Z,"List(List(item_813, 39.36, 1), List(item_112, 41.12, 4))","List(item_112, 41.12, 4)",item_112,41.12,4
event_1,1.753283167516559E9,2025-07-23 15:06:07,user_5484,click,2025-07-23T15:06:07.516493Z,"List(List(item_688, 21.68, 1), List(item_423, 22.82, 2), List(item_282, 93.46, 3))","List(item_688, 21.68, 1)",item_688,21.68,1
event_1,1.753283167516559E9,2025-07-23 15:06:07,user_5484,click,2025-07-23T15:06:07.516493Z,"List(List(item_688, 21.68, 1), List(item_423, 22.82, 2), List(item_282, 93.46, 3))","List(item_423, 22.82, 2)",item_423,22.82,2
event_1,1.753283167516559E9,2025-07-23 15:06:07,user_5484,click,2025-07-23T15:06:07.516493Z,"List(List(item_688, 21.68, 1), List(item_423, 22.82, 2), List(item_282, 93.46, 3))","List(item_282, 93.46, 3)",item_282,93.46,3
event_2,1.753283167516581E9,2025-07-23 15:06:07,user_6935,purchase,2025-07-23T15:06:07.516564Z,"List(List(item_828, 14.53, 2))","List(item_828, 14.53, 2)",item_828,14.53,2
event_3,1.753283167516599E9,2025-07-23 15:06:07,user_2627,purchase,2025-07-23T15:06:07.516584Z,"List(List(item_999, 96.23, 5))","List(item_999, 96.23, 5)",item_999,96.23,5
event_4,1.753283167516619E9,2025-07-23 15:06:07,user_4446,view,2025-07-23T15:06:07.516603Z,"List(List(item_487, 50.99, 5))","List(item_487, 50.99, 5)",item_487,50.99,5
event_5,1.753283167516643E9,2025-07-23 15:06:07,user_3345,view,2025-07-23T15:06:07.516623Z,"List(List(item_649, 16.13, 1), List(item_684, 48.76, 4), List(item_625, 84.04, 4))","List(item_649, 16.13, 1)",item_649,16.13,1
event_5,1.753283167516643E9,2025-07-23 15:06:07,user_3345,view,2025-07-23T15:06:07.516623Z,"List(List(item_649, 16.13, 1), List(item_684, 48.76, 4), List(item_625, 84.04, 4))","List(item_684, 48.76, 4)",item_684,48.76,4



&copy; 2025 Databricks, Inc. All rights reserved. Apache, Apache Spark, Spark, the Spark Logo, Apache Iceberg, Iceberg, and the Apache Iceberg logo are trademarks of the <a href="https://www.apache.org/" target="blank">Apache Software Foundation</a>.<br/>
<br/><a href="https://databricks.com/privacy-policy" target="blank">Privacy Policy</a> | 
<a href="https://databricks.com/terms-of-use" target="blank">Terms of Use</a> | 
<a href="https://help.databricks.com/" target="blank">Support</a>